# UFABC PDF to Spreadsheet

Define the ```url``` and ```filename```'s string variables that contain the URL of the PDF file to be converted and the new files name, repectively.

In [1]:
url = "http://prograd.ufabc.edu.br/pdf/turmas_salas_docentes_sa_2018.1.pdf"
filename = "2018.1_SA" # without extention
pdf_filename  = filename + '.pdf'
csv_filename  = filename + '.csv'
json_filename = filename + '.json'

Run [configuration notebook](config.ipynb).

In [2]:
%run config.ipynb

Import [functions notebook](functions.ipynb).

In [3]:
%run functions.ipynb

Download and preview the PDF file.

In [4]:
downloadPDF(url, pdf_filename)
IFrame(pdf_filename, 600, 300)

--2018-02-18 22:29:59--  http://prograd.ufabc.edu.br/pdf/turmas_salas_docentes_sa_2018.1.pdf
Resolving prograd.ufabc.edu.br (prograd.ufabc.edu.br)... 200.133.215.63
Connecting to prograd.ufabc.edu.br (prograd.ufabc.edu.br)|200.133.215.63|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689670 (674K) [application/pdf]
Saving to: ‘2018.1_SA.pdf’

2018.1_SA.pdf       100%[===================>] 673.51K   288KB/s    in 2.3s    

2018-02-18 22:30:02 (288 KB/s) - ‘2018.1_SA.pdf’ saved [689670/689670]



/home/jovyan/2018.1_SA.pdf

The PDF file is converted to a CSV file (*this might take a while*).

In [5]:
convertPDFtoCSV(pdf_filename, csv_filename)

Feb 18, 2018 10:30:03 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>


/home/jovyan/2018.1_SA.csv

The CSV file is then processed into a properly configured JSON file.

In [6]:
json_data = convertCSVtoJSON(csv_filename, json_filename)

/home/jovyan/2018.1_SA.json

Preview the JSON data.

In [7]:
showJSON(json_data)

Finally, process the JSON data into a spreadsheet.

In [8]:
sheet = convertJSONtoSheet(json_data)
sheet

{'id': 0, 'codigo': 'DAESZM035-17SA', 'subcodigo': 'ESZM035', 'disciplina': 'Aditivação de Polímeros', 'campus': 'Santo André', 'periodo': 'Diurno', 'turma': 'A', 'teoria': [{'id': 1, 'dia_da_semana': 'Segunda', 'horario_de_entrada': '10:00', 'horario_de_saida': '12:00', 'sala': 'S-301-3', 'frequencia': 'Semanal'}, {'id': 2, 'dia_da_semana': 'Quinta', 'horario_de_entrada': '08:00', 'horario_de_saida': '10:00', 'sala': 'S-301-3', 'frequencia': 'Semanal'}], 'pratica': [], 'docente_teoria': 'Anne Cristine Chinellato', 'docente_pratica': '', '0': 'Aditivação de Polímeros'}
{'id': 1, 'codigo': 'NA1MCTB001-17SA', 'subcodigo': 'MCTB001', 'disciplina': 'Álgebra Linear', 'campus': 'Santo André', 'periodo': 'Noturno', 'turma': 'A1', 'teoria': [{'id': 1, 'dia_da_semana': 'Segunda', 'horario_de_entrada': '19:00', 'horario_de_saida': '21:00', 'sala': 'S-204-0', 'frequencia': 'Semanal'}, {'id': 2, 'dia_da_semana': 'Quarta', 'horario_de_entrada': '21:00', 'horario_de_saida': '23:00', 'sala': 'S-204-0

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 15, 'minVisibleRows': 8}, precision=5)